In [26]:
import pandas as pd
from Bio import Entrez
from dotenv import load_dotenv
import os

load_dotenv()

True

### Currently working on the chest dataset only....will expand in iteration 2

In [7]:
article_metadata = pd.read_json("chest_dataset/article_metadata.json")
case_report_citations = pd.read_json("chest_dataset/case_report_citations.json")
cases_df = pd.read_csv("chest_dataset/cases.csv")
image_metadata = pd.read_json("chest_dataset/image_metadata.json", lines=True)

In [12]:
# Reading the data dictionary (consists of data info)

data_dict = pd.read_csv("medical_datasets/whole_multicare_dataset/data_dictionary.csv")

data_dict

,file,field,explanation
0,captions_and_labels.csv,file_id,Primary key for each row. Each row contains on...
1,captions_and_labels.csv,file,Name of the image file. The file is in the fol...
2,captions_and_labels.csv,main_image,Id from the original image (it corresponds to ...
3,captions_and_labels.csv,patient_id,"Id of the patient, created combining the PMC o..."
4,captions_and_labels.csv,license,License of the article. The possible values ar...
5,captions_and_labels.csv,caption,It is the caption that corresponds to the imag...
6,captions_and_labels.csv,chunk,List of chunks that were identified as having ...
7,captions_and_labels.csv,generic_label,List of labels that were assigned to each extr...
8,captions_and_labels.csv,pathology_test,If any chunk from the chunk list is of this ty...
9,captions_and_labels.csv,image_type,If any chunk from the chunk list is of this ty...


In [54]:
print(f"Article metadata consists of: {article_metadata.shape}")

article_metadata

Article metadata consists of: (75382, 5)


,pmcid,license,keywords,mesh_terms,major_mesh_terms
0,PMC3738355,CC BY-NC,"[vascular tumor, diagnostic imaging, liver, pa...",[Case Reports],[]
1,PMC5015624,CC BY,"[mri, aortic arch, carotid artery stenosis, ca...",[Case Reports],[]
2,PMC6381877,CC BY-NC,"[bevacizumab, egfr mutation, erlotinib, lung c...",[Case Reports],[]
3,PMC5912312,CC BY,"[acute coronary syndrome, diagnosis, fibromusc...",[],[]
4,PMC5287946,CC BY,None,"[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...","[Adenoma, Oxyphilic / pathology, Ki-67 Antigen..."
...,...,...,...,...,...
75377,PMC6180826,CC BY,None,[Case Reports],[]
75378,PMC3937567,CC BY-NC-ND,"[angiography and transcatheter embolization, a...",[Case Reports],[]
75379,PMC8200491,CC BY,"[rosai-dorfman disease, case report, pericardi...",[Case Reports],[]
75380,PMC7490131,CC BY,"[cryptic pregnancy, nigeria, delivery, denial ...","[Delivery, Obstetric, Denial, Psychological, O...","[Delivery, Obstetric, Denial, Psychological, O..."


In [55]:
print(f"Case report citations consists of: {case_report_citations.shape}")

case_report_citations

Case report citations consists of: (75382, 10)


,pmcid,doi,pmid,title,year,authors,journal,journal_detail,link,license
0,PMC3738355,10.1258/arsr.2012.120031,23986846,Littoral cell angioma mimicking hepatic tumor,2012,"[Wenhua Liang, Jingjing Lu, Mingwei Qin, Xinti...",Acta Radiol Short Rep,2012 Jul 31;1(6):arsr.2012.120031.,https://pubmed.ncbi.nlm.nih.gov/23986846/,CC BY-NC
1,PMC5015624,10.3205/oc000036,27625958,Central retinal artery occlusion following las...,2015,"[Payal J Shah, Brian Ellis, Lauren R DiGiovine...",GMS Ophthalmol Cases,2015 Dec 2;5:Doc14.,https://pubmed.ncbi.nlm.nih.gov/27625958/,CC BY
2,PMC6381877,10.1159/000493088,30792648,Promising Combination Therapy with Bevacizumab...,2019,"[Nobuhiko Seki, Maika Natsume, Ryosuke Ochiai,...",Case Rep Oncol,2019 Jan 21;12(1):91-97.,https://pubmed.ncbi.nlm.nih.gov/30792648/,CC BY-NC
3,PMC5912312,10.1177/2324709618770479,29707593,Spontaneous Coronary Artery Dissection: A Case...,2018,"[Ali Raza Ghani, Faisal Inayat, Nouman Safdar ...",J Investig Med High Impact Case Rep,2018 Apr 18;6:2324709618770479.,https://pubmed.ncbi.nlm.nih.gov/29707593/,CC BY
4,PMC5287946,10.1097/MD.0000000000005657,28121922,Malignant adenohypophysis spindle cell oncocyt...,2017,"[Xiangyi Kong, Dongmei Li, Yanguo Kong, Dingro...",Medicine (Baltimore),2017 Jan;96(4):e5657.,https://pubmed.ncbi.nlm.nih.gov/28121922/,CC BY
...,...,...,...,...,...,...,...,...,...,...
75377,PMC6180826,10.1259/bjrcr.20150149,30363580,Vanishing bone metastases: a pitfall in contra...,2015,"[R Pruente, C D Strickland, P J Koo]",BJR Case Rep,2015 Jul 3;1(3):20150149.,https://pubmed.ncbi.nlm.nih.gov/30363580/,CC BY
75378,PMC3937567,10.3402/jchimp.v4.22894,24596653,Acute mesenteric ischemia: a sequela of abdomi...,2014,"[Shiksha Kedia, Vijaya Raj Bhatt, Ashish Koira...",J Community Hosp Intern Med Perspect,2014 Feb 17;4(1).,https://pubmed.ncbi.nlm.nih.gov/24596653/,CC BY-NC-ND
75379,PMC8200491,10.3389/fcvm.2021.668031,34136545,Case Report: Pericardial Effusion Treated With...,2021,"[Edoardo Conte, Antonio Brucato, Francesco Pet...",Front Cardiovasc Med,2021 May 31;8:668031.,https://pubmed.ncbi.nlm.nih.gov/34136545/,CC BY
75380,PMC7490131,10.11604/pamj.2020.36.205.23790,32963671,Unexpected delivery: a case report of cryptic ...,2020,"[Uloaku Akubueze Nto-Ezimah, Nto Johnson Nto, ...",Pan Afr Med J,2020 Jul 22;36:205.,https://pubmed.ncbi.nlm.nih.gov/32963671/,CC BY


In [57]:
case_report_citations['title'].unique()

array(['Littoral cell angioma mimicking hepatic tumor',
       'Central retinal artery occlusion following laser treatment for ocular ischemic aortic arch syndrome',
       'Promising Combination Therapy with Bevacizumab and Erlotinib in an EGFR-Mutated NSCLC Patient with MET Amplification Who Showed Intrinsic Resistance to Initial EGFR-TKI Therapy',
       ...,
       'Case Report: Pericardial Effusion Treated With Pericardiectomy Plus Right Atrial Mass Resection: A 2-Year Follow-Up of Cardiac Rosai-Dorfman Disease',
       'Unexpected delivery: a case report of cryptic pregnancy in Nigeria',
       'Bell clapper testis, torsion, and detorsion: a case report'],
      dtype=object)

In [15]:
print(f"All cases consists of: {cases_df.shape}")

cases_df.head()

All cases consists of: (96428, 5)


,case_id,pmcid,gender,age,case_text
0,PMC3738355_01,PMC3738355,Female,53.0,A 53-year-old woman presented with a 10-year h...
1,PMC5015624_01,PMC5015624,Female,69.0,A 69-year-old Caucasian female with coronary a...
2,PMC6381877_01,PMC6381877,Male,60.0,A 60-year-old male smoker presented with persi...
3,PMC5912312_01,PMC5912312,Female,41.0,A 41-year-old female with a past medical histo...
4,PMC5912312_02,PMC5912312,Male,51.0,A 51-year-old male with a history of SCAD pres...


In [20]:
print(cases_df['case_text'][0])

A 53-year-old woman presented with a 10-year history of intermittent abdominal pain, swelling and continuous vomiting. The patient denied presence of fever, nausea, and weight loss. There were no significant findings at physical examination. An abdominal ultrasound exam revealed a 10.4 x 10.0 cm mass of heterogeneous echogenicity in the left upper abdomen. Axial unenhanced CT scan (Fig. 1a) confirmed an ill-defined heterogeneous mass 9.1 cm in diameter that filled the left abdominal quadrant, located between the left lobe of the liver and the spleen. The CT attenuation of the mass was around 26-53 HU on non-enhanced scan. After intravenous contrast medium injection the mass enhanced gradually and heterogeneously (Fig. 1b-e), but was still hypodense relative to the spleen. There was no accompanying lymphadenopathy or evidence of malignant process elsewhere in the abdomen. Because the origin of the mass was unknown and a primary malignancy could not be excluded, the patient underwent an 

In [59]:
print(f"Image metadata consists of: {image_metadata.shape}")

image_metadata.head()

Image metadata consists of: (2287, 9)


,file_id,file_path,normalized_extractions,labels,caption,raw_image_link,case_id,license,split_during_preprocessing
0,file_000048,medical_datasets/chest_dataset/images/PMC1/PMC...,"[x_ray, chest, lung, right]","[Site, Imaging_Test, Problem, Laterality, Site]",Chest radiograph on admission showing pleural ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,PMC10006502_01,CC BY-NC-ND,True
1,file_000111,medical_datasets/chest_dataset/images/PMC1/PMC...,"[x_ray, chest, lung]","[Site, Imaging_Test, Site, Imaging_Finding]",(A): Chest x-ray on Day 1 revealed multiple lu...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,PMC10009051_01,CC BY-NC-ND,False
2,file_000119,medical_datasets/chest_dataset/images/PMC1/PMC...,"[posteroanterior, chest, pathological_finding,...","[Site, Imaging_Test, Imaging_Technique, Latera...",Chest X-ray posterior-anterior view showing bi...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,PMC10009052_01,CC BY,True
3,file_000167,medical_datasets/chest_dataset/images/PMC1/PMC...,"[x_ray, chest, lung, right]","[Site, Imaging_Test, Imaging_Finding, Laterali...",A chest x-ray image showed a ground glass appe...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,PMC10010120_01,CC BY-NC,True
4,file_000298,medical_datasets/chest_dataset/images/PMC1/PMC...,"[chest, ct, contrast, soft_tissue_window, x_ra...","[Site, Imaging_Test, Imaging_Technique, Imagin...","Both preoperative chest X-ray ,axial CT-scan w...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,PMC10011863_01,CC BY-NC-ND,False


### Clearly, image metadata comprises mostly of chest data as compared to other which consist of a lot more data ( including tumour as well). So will need to only use the case which has a xray scan for now 